In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import os

In [ ]:
import requests
from xml.etree import ElementTree as ET
from datetime import datetime

# API endpoint
url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst"

# API key
service_key = "2tvkzWa%2FDa5wAH2C4F09CShggc7fixR0Jowlz4jhXWSCwlVk%2BAbL2Yxt7QmsCKRf0ijOA0IxzzNPHTQeLhxixQ%3D%3D"

# Today's date
today = datetime.today().strftime("%Y%m%d")

# Parameters
params = {
    "serviceKey": service_key,
    "numOfRows": 10,
    "pageNo": 1,
    "dataType": "XML",
    "base_date": today,
    "base_time": "0500",
    "nx": "60",  # Seoul's grid x
    "ny": "127"  # Seoul's grid y
}

# Send request
response = requests.get(url, params=params)

# Parse XML response
root = ET.fromstring(response.content)

# Find temperature data
temps = [float(item.find('fcstValue').text) for item in root.iter('item') if item.find('category').text == 'T3H']

# Calculate average temperature
avg_temp = sum(temps) / len(temps)

In [2]:
user_id = 1
today = 21.5
items_id = "2, 17, 28, 36, 45, 51"

In [3]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(male)/outfit(male).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-30.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일']].copy()
df_temp = df_weather[['일시', '평균기온(°C)']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_temp['일시'] = pd.to_datetime(df_temp['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_temp, left_on='작성일', right_on='일시').drop('일시', axis=1)

'''df_merged'''

'df_merged'

In [4]:
# 평균기온(°C) column의 최대값과 최솟값
max_temp = df_merged['평균기온(°C)'].max()
min_temp = df_merged['평균기온(°C)'].min()

In [5]:
bins=np.round(np.arange(min_temp -5, max_temp+5, 5), 1)
labels=np.arange(0, (max_temp-min_temp)//5+2)
today_class = pd.cut([today], bins=bins, labels=labels).astype('float64')

In [6]:
today_class

array([7.])

In [10]:
path = f'../data/satisfaction/CF/male/user_{user_id}/satifaction.csv'

df_satisfaction = pd.read_csv(path)

In [14]:
df_satisfaction

,옷 id,예측값
0,"2, 17, 28, 36, 45, 51",4.0
1,"2, 17, 28, 29, 36, 45",5.0
2,"1, 14, 28, 36, 45",6.0
3,"2, 14, 28, 36",5.0
4,"2, 14, 28, 29, 36",3.0
...,...,...
691,"6, 18, 25, 35",3.0
692,"6, 17, 25, 35",4.0
693,"6, 17, 27, 35",3.0
694,"6, 18, 34",4.0


In [22]:
item_satis = df_satisfaction[df_satisfaction['옷 id'] == "2, 17, 28, 36, 45, 51"]['예측값'].values
pred_satis = today_class - item_satis
pred_satis

array([3.])

In [23]:
if pred_satis >= 2:
    print("VERY_HOT")
elif pred_satis >= 1 and pred_satis < 2:
    print("HOT")
elif pred_satis >= -1 and pred_satis < 1:
    print("NORMAL")
elif pred_satis >= -2 and pred_satis < -1:
    print("COLD")
else:
    print("VERY_COLD")

VERY_HOT
